In [ ]:
# i have a .txt with the text split up into cantos, but I'm going to try to rip the text from the Dartmouth Dante Project (DDP)
# it's easy to find the text in the HTML editor, but the webpage updates when you 
# go to the next canto/cantica. It should be easy enough to automate though
# Here is the link to Inferno I: http://dantelab.dartmouth.edu/reader?reader%5Bcantica%5D=1&reader%5Bcanto%5D=1
# 'cantica%5D=1' gives Inferno, =2 gives Purgatorio, and =3 gives Paradiso
# 'canto%5D=1' gives Canto 1, =2, gives Canto 2, etc.

# just a little test

In [6]:
## Import base packages we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

from seaborn import set_style
set_style("whitegrid")

import bs4
print(bs4.__version__)

from bs4 import BeautifulSoup

import requests


4.12.2


In [7]:
# for listing the cantos 1-100 instead of e.g. Inferno V = (1,5) and Paradiso XXX = (3,30)
def canto_conversion(i,j):
    if i==0:
        return j+1
    elif i==1:
        return 34+j+1
    else:
        return 67+j+1

In [8]:
comedy_text = [""]*100

# cycle through all cantos
for i in range(3):
    for j in range(34):
        if (i==1 and j==33) or (i==2 and j==33): #there is inf. 34, but no purg. 34 nor par. 34
            continue
        url = "http://dantelab.dartmouth.edu/reader?reader%5Bcantica%5D=" + str(i+1) + "&reader%5Bcanto%5D=" + str(j+1)
        response = requests.get(url=url)
        
        # the relevant text is found in a division id'd by 'L1' in an element called 'pre' with the class 'commentarytercet'
        soup = BeautifulSoup(response.content, 'html.parser').find("div", {"id": "L1"})
        canto_text_container = soup.find_all('pre', attrs={'class': 'commentarytercet'})
        
        # cleaning up the text
        canto_text_container_string = [str(txt) for txt in canto_text_container]
        for k in range(len(canto_text_container_string)):
            canto_text_container_string[k]=canto_text_container_string[k].replace('<i>','').replace('</i>','').replace(',','').replace('.','').replace(';','').replace('?','').replace('!','').replace(':','').replace('“','').replace('”','')
            
        # re-souping and parsing
        canto_text_clean = ""
        for k in range(len(canto_text_container_string)):
            soup = BeautifulSoup(canto_text_container_string[k], 'html.parser')
            for br in soup.find_all('br'):
                if br.nextSibling != None:
                    canto_text_clean += str(br.nextSibling)+'\n'
                    
        # store the text in an array
        comedy_text[canto_conversion(i,j)-1] = canto_text_clean
        sleep(0.5)

In [12]:
dante_word_dict = {"word" : [], "verse_num" : [], "cantica" : [], "canto" : [], "rhyme?" : []}
dante_canto_dict = {'cantica' : [""]*100, 'canto' : [0]*100, 'num_verses' : [0]*100, 'num_words' : [0]*100 }
for i in range(100):
    #first deal with dante_canto_dict, since that's easy
    if (i <= 33):
        dante_canto_dict['cantica'][i]="Inferno"
        dante_canto_dict['canto'][i]=i+1
    elif (i <= 66):
        dante_canto_dict['cantica'][i]="Purgatorio"
        dante_canto_dict['canto'][i]=i-33
    else:
        dante_canto_dict['cantica'][i]="Paradiso"
        dante_canto_dict['canto'][i]=i-66
    
    verse_by_verse = comedy_text[i].splitlines()
    dante_canto_dict['num_verses'][i]=len(verse_by_verse)
    
    
    words, verse_nums, cantica, canto, rhyme = [], [], [], [], []
    word_num = 0
    verse_num = 1 #count verse number
    for verse in verse_by_verse:
        for word in verse.split():
            words.append(word)
            verse_nums.append(verse_num)
            if (i <= 33):
                cantica.append("Inferno")
                canto.append(i+1)
            elif (i <= 66):
                cantica.append("Purgatorio")
                canto.append(i-33)
            else:
                cantica.append("Paradiso")
                canto.append(i-66)
            if word == verse.split()[-1]:
                rhyme.append(1)
            else:
                rhyme.append(0)
            word_num += 1
        verse_num += 1
    dante_word_dict["word"] += words
    dante_word_dict["verse_num"] += verse_nums
    dante_word_dict["cantica"] += cantica
    dante_word_dict["canto"] += canto
    dante_word_dict["rhyme?"] += rhyme
    dante_canto_dict['num_words'][i]=word_num

In [15]:
df = pd.DataFrame(dante_canto_dict)
df.to_csv("../data/commedia_cantos.csv", index=False)